In [1]:
import os
import sys
import numpy as np
import matplotlib.pyplot as plt
import cv2
from tqdm import tqdm
sys.path.append('../../../src')

from common import common, transform, json
from common.json import GA_FORMAT
from group_activity.group_activity import GroupActivity

In [2]:
room_num = '09'
date = '20210304'
name = 'pass1'

In [9]:
group_activity_json = os.path.join(
    common.data_dir, '{0}/{1}/{2}/json/group_activity.json'.format(room_num, date, name))
field_path = os.path.join(
    common.data_dir, '{}/field0304.png'.format(room_num))

In [10]:
# homography
field_raw = cv2.imread(field_path)
p_video = common.homo[room_num + '_0304'][0]
p_field = common.homo[room_num + '_0304'][1]
homo = transform.Homography(p_video, p_field, field_raw.shape)

In [11]:
ga_data = json.load(group_activity_json)

In [12]:
def make_group_activity(data, method):
    ga = GroupActivity(field_raw, method)
    ga.indicator_dict[method] = data[method]
    return ga
ga = make_group_activity(ga_data, 'passing')

In [ ]:
preds = []
pre_pred = 0
last_true_frame = 0
for data in ga.indicator_dict[method]:
    persons = data[GA_FORMAT[method][1]]
    if persons[0] in target_persons[name] and persons[1] in target_persons[name]:
        frame_num = data[GA_FORMAT[method][0]]
        pred = data[GA_FORMAT[method][3]]
        preds.append(pred)

        if pre_pred == 0 and pred == 1:
            if frame_num - last_true_frame < 50:
                print(last_true_frame)
                print(frame_num)
                # 20フレーム以内に 1 が来たら間を穴埋めする
                for i in range(last_true_frame, frame_num + 1):
                    preds[i] = 1

        if pred == 1:
            last_true_frame = frame_num
            
        pre_pred = pred